# AI Trading Assistant - Usage Examples
This notebook demonstrates how to use the various modules of the AI Trading Assistant.

**Team Members:**
- Prem Pratap (22070126078)
- Punit Chetwani (22070126079)
- Zaheer Khan (22070126066)

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '../src/lib')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from data_fetcher import StockDataFetcher
from ml_models import LSTMPredictor, ProphetPredictor
from portfolio_optimizer import PortfolioOptimizer
from risk_analyzer import RiskAnalyzer
from technical_indicators import TechnicalIndicators
from sentiment_analyzer import SentimentAnalyzer
from visualizations import FinancialVisualizer

## 1. Data Fetching

In [ ]:
# Initialize data fetcher
fetcher = StockDataFetcher()

# Fetch stock data
ticker = 'AAPL'
data = fetcher.get_stock_data(ticker, period='1y')

print(f"Fetched {len(data)} records for {ticker}")
data.head()

## 2. Technical Indicators

In [ ]:
# Calculate technical indicators
signals = TechnicalIndicators.generate_trading_signals(data)

# Display latest signals
print("Latest Trading Signals:")
print(f"RSI: {signals['RSI'].iloc[-1]:.2f}")
print(f"MACD: {signals['MACD'].iloc[-1]:.4f}")
print(f"Overall Signal: {signals['Overall_Signal'].iloc[-1]}")

signals.tail()

## 3. Risk Analysis

In [ ]:
# Initialize risk analyzer
analyzer = RiskAnalyzer(risk_free_rate=0.02)

# Get comprehensive risk report
risk_report = analyzer.comprehensive_risk_report(data)

print("Risk Metrics:")
for metric, value in risk_report.items():
    if isinstance(value, (int, float)):
        print(f"{metric}: {value:.4f}")

## 4. Portfolio Optimization

In [ ]:
# Fetch data for multiple stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
stock_data = {}

for ticker in tickers:
    stock_data[ticker] = fetcher.get_stock_data(ticker, period='1y')

# Initialize optimizer
optimizer = PortfolioOptimizer(risk_free_rate=0.02)
returns = optimizer.prepare_data(stock_data)

# Find optimal portfolio
max_sharpe = optimizer.optimize_sharpe()

print("\nMaximum Sharpe Ratio Portfolio:")
print(f"Expected Return: {max_sharpe['return']*100:.2f}%")
print(f"Volatility: {max_sharpe['volatility']*100:.2f}%")
print(f"Sharpe Ratio: {max_sharpe['sharpe_ratio']:.4f}")
print("\nAllocation:")
for ticker, weight in max_sharpe['weights'].items():
    print(f"{ticker}: {weight*100:.2f}%")

## 5. Price Prediction with LSTM

In [ ]:
# Note: This requires TensorFlow to be installed
try:
    # Initialize LSTM model
    lstm_model = LSTMPredictor(lookback=60, units=50)
    
    # Train model
    print("Training LSTM model...")
    metrics = lstm_model.train(data, epochs=10, batch_size=32)
    
    print("\nTraining Metrics:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"R² Score: {metrics['r2']:.4f}")
    
    # Make predictions
    predictions = lstm_model.predict(data, days=30)
    print("\nPredictions for next 5 days:")
    print(predictions.head())
    
except ImportError:
    print("TensorFlow not installed. Skipping LSTM example.")

## 6. Sentiment Analysis

In [ ]:
# Initialize sentiment analyzer
sentiment = SentimentAnalyzer()

# Sample headlines
headlines = [
    "Apple stock surges on strong earnings report",
    "Tech sector faces regulatory concerns",
    "Apple announces breakthrough innovation",
    "Market volatility continues amid uncertainty",
    "Apple maintains steady growth trajectory"
]

# Analyze sentiment
sentiment_df = sentiment.analyze_headlines(headlines)
agg_sentiment = sentiment.aggregate_sentiment(sentiment_df)

print("Sentiment Analysis Results:")
print(f"Overall Mood: {agg_sentiment['overall_mood']}")
print(f"Average Sentiment: {agg_sentiment['average_sentiment']:.3f}")
print(f"Positive Ratio: {agg_sentiment['positive_ratio']*100:.1f}%")

sentiment_df

## 7. Visualizations

In [ ]:
# Initialize visualizer
viz = FinancialVisualizer()

# Create candlestick chart
fig1 = viz.plot_candlestick(data, f"{ticker} Price Chart")
fig1.show()

# Create technical indicators chart
fig2 = viz.plot_technical_indicators(signals, f"{ticker} Technical Analysis")
fig2.show()

# Create portfolio allocation chart
fig3 = viz.plot_portfolio_allocation(
    max_sharpe['weights'],
    "Optimal Portfolio Allocation"
)
fig3.show()

## 8. Complete Analysis Pipeline

In [ ]:
def analyze_stock(ticker, period='1y'):
    """Complete analysis pipeline for a stock."""
    
    print(f"\n{'='*60}")
    print(f"Complete Analysis for {ticker}")
    print(f"{'='*60}\n")
    
    # 1. Fetch data
    fetcher = StockDataFetcher()
    data = fetcher.get_stock_data(ticker, period=period)
    
    if data.empty:
        print("No data available")
        return
    
    # 2. Calculate technical indicators
    signals = TechnicalIndicators.generate_trading_signals(data)
    latest_signal = signals.iloc[-1]
    
    print("1. TRADING SIGNALS")
    print(f"   Overall Signal: {latest_signal['Overall_Signal']}")
    print(f"   RSI: {latest_signal['RSI']:.2f}")
    print(f"   MACD Signal: {latest_signal['MACD_Signal']}")
    
    # 3. Risk analysis
    analyzer = RiskAnalyzer()
    risk_report = analyzer.comprehensive_risk_report(data)
    
    print("\n2. RISK METRICS")
    print(f"   Volatility: {risk_report['volatility']*100:.2f}%")
    print(f"   Sharpe Ratio: {risk_report['sharpe_ratio']:.4f}")
    print(f"   Max Drawdown: {risk_report['max_drawdown']*100:.2f}%")
    print(f"   VaR (95%): {risk_report['var_95']*100:.2f}%")
    
    # 4. Performance metrics
    returns = data['Close'].pct_change().dropna()
    cumulative_return = (1 + returns).prod() - 1
    
    print("\n3. PERFORMANCE")
    print(f"   Cumulative Return: {cumulative_return*100:.2f}%")
    print(f"   Annualized Return: {returns.mean()*252*100:.2f}%")
    
    print(f"\n{'='*60}\n")

# Analyze a stock
analyze_stock('AAPL')

## Conclusion

This notebook demonstrates the key functionalities of the AI Trading Assistant:
- Stock data fetching
- Technical indicator calculation
- Risk analysis
- Portfolio optimization
- Price prediction
- Sentiment analysis
- Interactive visualizations

For the full application with an interactive UI, run the Streamlit app:
```bash
streamlit run src/main.py
```